#########################################################<br>
 AIR CAVITY<br>
 IMPEDANCE PAROI<br>
 THIN FLEXIBLE STRUCTURE<br>
 XFEM<br>
#########################################################

#########################################################<br>
Libraries<br>
#########################################################

In [1]:
import string
import time
import scipy
import scipy.sparse
import scipy.sparse.linalg
import numpy as np

In [2]:
import pickle

In [3]:
import sys
sys.path.append('../librairies')

In [4]:
import silex_lib_xfem_acou_tet4
import silex_lib_gmsh
import silex_lib_dkt_fortran as silex_lib_dkt

In [5]:
import silex_lib_porous_tet4_fortran

In [6]:
import mumps

In [7]:
from mpi4py import MPI
comm = MPI.COMM_WORLD
nproc=comm.Get_size()
rank = comm.Get_rank()

In [8]:
class comm_mumps_one_proc:
    rank = 0
    def py2f(self):
        return 0
mycomm=comm_mumps_one_proc()

#########################################################<br>
To run it in parallel for several frequencies:<br>
export OPENBLAS_NUM_THREADS=1<br>
mpirun -np 20  python3 Main_xfem_fluid_porous_flex_struc_no_reduction_8.py<br>
<br>
To run it in sequentiel frequency per frequency with openblas in parrallel:<br>
export OPENBLAS_NUM_THREADS=10<br>
python3 Main_toto.py<br>
<br>
#########################################################

In [9]:
if rank==0:
    print ("time at the beginning of the computation:",time.ctime())

time at the beginning of the computation: Wed Apr 13 10:20:02 2022


############################################################<br>
############################################################<br>
             S T A R T   M A I N   P R O B L E M<br>
############################################################<br>
############################################################

############################################################<br>
Datas<br>
############################################################

In [10]:
mesh_file='geom/cavity12_with_porous_air'
results_file='results/cavity12_with_impedance_air_flexible_structure'

In [11]:
flag_write_gmsh_results=1

In [12]:
freq_ini     = 10.0
freq_end     = 200.0
nb_freq_step_per_proc=800

In [13]:
nb_freq_step = nb_freq_step_per_proc*nproc
deltafreq=(freq_end-freq_ini)/(nb_freq_step-1)

air

In [14]:
celerity=343.0 # ok
rho=1.21 # ok

shell structure

In [15]:
material_Struc=[]
material_Struc.append(75000.0e6) # E Young
material_Struc.append(0.33) # nu
material_Struc.append(5.0e-3) # thickness
material_Struc.append(2700.0) # rho

impedance paroi : article Walid-JFD : CMAME 2008

In [16]:
d_imp_paroi = 50.0 # Pa.s/m
k_imp_paroi = 5.0e6 # Pa/m

############################################################<br>
Load fluid mesh<br>
############################################################

In [17]:
tic = time.clock()

/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [18]:
fluid_nodes    = silex_lib_gmsh.ReadGmshNodes(mesh_file+'.msh',3)
fluid_elements1,IdNodes1 = silex_lib_gmsh.ReadGmshElements(mesh_file+'.msh',4,1) # air, cavity + controlled volume
fluid_elements5,IdNodes5 = silex_lib_gmsh.ReadGmshElements(mesh_file+'.msh',4,5) # air, ONLY controlled volume
fluid_elements_S3,IdNodesS3 = silex_lib_gmsh.ReadGmshElements(mesh_file+'.msh',2,3)# air-porous interface surface : impedance paroi
#fluid_elements2,IdNodes2 = silex_lib_gmsh.ReadGmshElements(mesh_file+'.msh',4,2) # porous, volume
#fluid_elements_S4,IdNodesS4 = silex_lib_gmsh.ReadGmshElements(mesh_file+'.msh',2,4) # porous, external surface

In [19]:
fluid_nnodes   = fluid_nodes.shape[0]
fluid_nelem1   = fluid_elements1.shape[0]
#fluid_nelem2   = fluid_elements2.shape[0]
fluid_nelem3   = fluid_elements_S3.shape[0]
#fluid_nelem4   = fluid_elements_S4.shape[0]
fluid_nelem5   = fluid_elements5.shape[0]

In [20]:
fluid_nnodes1 = IdNodes1.shape[0]
#fluid_nnodes2 = IdNodes2.shape[0]
fluid_nnodes3 = IdNodesS3.shape[0]
#fluid_nnodes4 = IdNodesS4.shape[0]
fluid_nnodes5 = IdNodes5.shape[0]

In [21]:
fluid_ndof1     = fluid_nnodes1

In [22]:
if rank==0:
    print ("Number of nodes:",fluid_nnodes)
    print ("Number of elements in air:",fluid_nelem1)
    print ("Number of nodes in air:",fluid_nnodes1)
    print ("Number of nodes at interface:",fluid_nnodes3)

Number of nodes: 23382
Number of elements in air: 118211
Number of nodes in air: 21699
Number of nodes at interface: 407


renumbering air

In [23]:
old = np.unique(fluid_elements1)
new = list(range(1,len(np.unique(fluid_elements1))+1))
new_nodes=fluid_nodes[np.unique(fluid_elements1)-1,:]

In [24]:
dico1 = dict(zip(old,new))
new_elements=np.zeros((fluid_nelem1,4),dtype=int)
for e in range(fluid_nelem1):
    for i in range(4):
        new_elements[e,i]=dico1[fluid_elements1[e][i]]

In [25]:
fluid_elements1 = new_elements
fluid_nodes1    = new_nodes

In [26]:
new_elements=np.zeros((fluid_nelem5,4),dtype=int)
for e in range(fluid_nelem5):
    for i in range(4):
        new_elements[e,i]=dico1[fluid_elements5[e][i]]

In [27]:
fluid_elements5 = new_elements

# renumbering porous<br>
old = np.unique(fluid_elements2)<br>
new = list(range(1,len(np.unique(fluid_elements2))+1))<br>
new_nodes=fluid_nodes[np.unique(fluid_elements2)-1,:]<br>
<br>
dico2 = dict(zip(old,new))<br>
new_elements=np.zeros((fluid_nelem2,4),dtype=int)<br>
for e in range(fluid_nelem2):<br>
    for i in range(4):<br>
        new_elements[e,i]=dico2[fluid_elements2[e][i]]<br>
<br>
fluid_elements2 = new_elements<br>
fluid_nodes2    = new_nodes

# renumbering porous external surfaces<br>
for i in range(len(IdNodesS4)):<br>
    IdNodesS4[i]=dico2[IdNodesS4[i]]

# Boundary conditions on air cavity<br>
IdNodesFixed_porous_us_x=IdNodesS4<br>
##IdNodesFixed_porous_us_y=np.unique(np.hstack([IdNodesS4,IdNodesS6]))<br>
IdNodesFixed_porous_us_y=IdNodesS4<br>
IdNodesFixed_porous_us_z=IdNodesS4<br>
IdNodesFixed_porous_uf_x=IdNodesS4<br>
IdNodesFixed_porous_uf_y=IdNodesS4<br>
IdNodesFixed_porous_uf_z=IdNodesS4<br>
<br>
Fixed_Dofs_porous = np.hstack([(IdNodesFixed_porous_us_x-1)*6,<br>
                                  (IdNodesFixed_porous_us_y-1)*6+1,<br>
                                  (IdNodesFixed_porous_us_z-1)*6+2,<br>
                                  (IdNodesFixed_porous_uf_x-1)*6+3,<br>
                                  (IdNodesFixed_porous_uf_y-1)*6+4,<br>
                                  (IdNodesFixed_porous_uf_z-1)*6+5<br>
                                  ])

get connectivity at air- impedance paroi

In [28]:
IdNodesS3_for_1=np.zeros(fluid_nnodes3,dtype=int)
#IdNodesS3_for_2=np.zeros(fluid_nnodes3,dtype=int)
for i in range(fluid_nnodes3):
    IdNodesS3_for_1[i]=dico1[IdNodesS3[i]] # for the air mesh
    #IdNodesS3_for_2[i]=dico2[IdNodesS3[i]] # for the porous mesh

In [29]:
InterfaceConnectivity=np.zeros((fluid_nelem3,3),dtype=int)
for e in range(fluid_nelem3):
    for i in range(3):
        InterfaceConnectivity[e,i]   = dico1[fluid_elements_S3[e,i]] # for the air mesh
##        InterfaceConnectivity[e,i+3] = dico2[fluid_elements_S3[e,i]] # for the porous mesh

In [30]:
if (flag_write_gmsh_results==1) and (rank==0):
    silex_lib_gmsh.WriteResults(results_file+'_air_cavity_Mesh1',fluid_nodes1,fluid_elements1,4)
    #silex_lib_gmsh.WriteResults(results_file+'_porous_material_Mesh2',fluid_nodes2,fluid_elements2,4)
    silex_lib_gmsh.WriteResults(results_file+'_porous_air_interface_Mesh_surface3',fluid_nodes,fluid_elements_S3,2)
    #silex_lib_gmsh.WriteResults(results_file+'_porous_fixed_Mesh_surface4',fluid_nodes,fluid_elements_S4,2)
    silex_lib_gmsh.WriteResults(results_file+'_air_controlled_volume_Mesh5',fluid_nodes1,fluid_elements5,4)

############################################################<br>
Load structure mesh<br>
############################################################

In [31]:
struc_nodes        = silex_lib_gmsh.ReadGmshNodes(mesh_file+'_struc.msh',3)
struc_elements,tmp = silex_lib_gmsh.ReadGmshElements(mesh_file+'_struc.msh',2,6)
struc_boun,tmp     = silex_lib_gmsh.ReadGmshElements(mesh_file+'_struc.msh',1,7)

In [32]:
struc_nnodes   = struc_nodes.shape[0]
struc_nelem    = struc_elements.shape[0]
struc_ndof     = struc_nnodes*6

In [33]:
if (flag_write_gmsh_results==1) and (rank==0):
    silex_lib_gmsh.WriteResults2(results_file+'_struc_mesh_surface_6',struc_nodes,struc_elements,2)
    silex_lib_gmsh.WriteResults2(results_file+'_struc_boun_mesh_line_7',struc_nodes,struc_boun,1)

In [34]:
if rank==0:
    print ("nnodes for structure=",struc_nnodes)
    print ("nelem for structure=",struc_nelem)

nnodes for structure= 3179
nelem for structure= 6143


Find the fixed dofs and the free dofs of the structure

In [35]:
tmp=scipy.sparse.find(struc_nodes[:,2]==0.0)# z=0
FixedStrucNodes=tmp[1]+1

In [36]:
FixedStrucDofUx=(FixedStrucNodes-1)*6
FixedStrucDofUy=(FixedStrucNodes-1)*6+1
FixedStrucDofUz=(FixedStrucNodes-1)*6+2
FixedStrucDofRx=(FixedStrucNodes-1)*6+3
FixedStrucDofRy=(FixedStrucNodes-1)*6+4
FixedStrucDofRz=(FixedStrucNodes-1)*6+5
#FixedStrucDofRx=[]
#FixedStrucDofRy=[]
#FixedStrucDofRz=[]

In [37]:
FixedStrucDof=np.hstack([FixedStrucDofUx,FixedStrucDofUy,FixedStrucDofUz,FixedStrucDofRx,FixedStrucDofRy,FixedStrucDofRz])

In [38]:
SolvedDofS=np.setdiff1d(range(struc_ndof),FixedStrucDof)

In [39]:
FS=np.zeros(struc_ndof)
#IddofLoadStructure=[(IdNodeLoadStructure-1)*6+2]
#FS[IddofLoadStructure]=1.0

############################################################<br>
Compute structure matrices<br>
############################################################

In [40]:
tic = time.clock()

/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [41]:
IIks,JJks,Vks,Vms=silex_lib_dkt.stiffnessmatrix(struc_nodes,struc_elements,material_Struc)

In [42]:
KSS = scipy.sparse.csc_matrix( (Vks,(IIks,JJks)), shape=(struc_ndof,struc_ndof) )
MSS = scipy.sparse.csc_matrix( (Vms,(IIks,JJks)), shape=(struc_ndof,struc_ndof) )

In [43]:
toc = time.clock()
if rank==0:
    print ("time for computing structure:",toc-tic)

time for computing structure: 0.19110100000000063


/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


################################################################<br>
compute level set<br>
################################################################

In [44]:
tic = time.clock()

/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [45]:
LevelSet,distance = silex_lib_xfem_acou_tet4.computelevelset(fluid_nodes1,struc_nodes,struc_elements)

In [46]:
toc = time.clock()
if rank==0:
    print ("time to compute level set:",toc-tic)

time to compute level set: 0.29857200000000006


/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [47]:
tic = time.clock()

/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [48]:
tangent_nodes,tangent_mesh=silex_lib_xfem_acou_tet4.buildtangentedgemesh(struc_nodes,struc_elements,struc_boun)

In [49]:
LevelSetTangent,tmp = silex_lib_xfem_acou_tet4.computelevelset(fluid_nodes1,tangent_nodes,tangent_mesh)

In [50]:
toc = time.clock()
if rank==0:
    print ("time to compute tangent level set:",toc-tic)

time to compute tangent level set: 0.051736000000000004


/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [51]:
if (flag_write_gmsh_results==1) and (rank==0):
    silex_lib_gmsh.WriteResults2(results_file+'_LS_signed_distance',fluid_nodes1,fluid_elements1,4,[[[LevelSet],'nodal',1,'Level set']])
    silex_lib_gmsh.WriteResults2(results_file+'_LS_tangent_level_set',fluid_nodes1,fluid_elements1,4,[[[LevelSetTangent],'nodal',1,'Tangent level set']])
    silex_lib_gmsh.WriteResults2(results_file+'_LS_distance',fluid_nodes1,fluid_elements1,4,[[[distance],'nodal',1,'Distance']])
    silex_lib_gmsh.WriteResults2(results_file+'_LS_tangent_mesh',tangent_nodes,tangent_mesh,2)

################################################################<br>
Get enriched nodes and elements<br>
################################################################

In [52]:
tic = time.clock()

/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [53]:
LSEnrichedElements,NbLSEnrichedElements=silex_lib_xfem_acou_tet4.getenrichedelementsfromlevelset(fluid_elements1,LevelSet)
LSEnrichedElements=LSEnrichedElements[list(range(NbLSEnrichedElements))]
EnrichedElements,NbEnrichedElements=silex_lib_xfem_acou_tet4.getsurfenrichedelements(struc_nodes,struc_elements,fluid_nodes1,fluid_elements1[LSEnrichedElements])
EnrichedElements=np.unique(EnrichedElements[list(range(NbEnrichedElements))])
EnrichedElements=LSEnrichedElements[EnrichedElements-1]
toc = time.clock()
if rank==0:
    print ("time to find surface enriched elements:",toc-tic)

time to find surface enriched elements: 2.802792


/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [54]:
tic = time.clock()

/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [55]:
EdgeEnrichedElements,nbenrelts = silex_lib_xfem_acou_tet4.getedgeenrichedelements(struc_nodes,struc_boun,fluid_nodes1,fluid_elements1)
EdgeEnrichedElements=np.unique(EdgeEnrichedElements[list(range(nbenrelts))])-1

In [56]:
EdgeEnrichedElementsInAllMesh,nbEdgeEnrichedElementsInAllMesh=silex_lib_xfem_acou_tet4.getenrichedelementsfromlevelset(fluid_elements1,LevelSetTangent)
EdgeEnrichedElementsInAllMesh=np.unique(EdgeEnrichedElementsInAllMesh[list(range(nbEdgeEnrichedElementsInAllMesh))])

In [57]:
toc = time.clock()
if rank==0:
    print ("time to find edge enriched elements:",toc-tic)

time to find edge enriched elements: 0.324935


/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [58]:
HeavisideEnrichedElements=np.setdiff1d(EnrichedElements,EdgeEnrichedElements)

In [59]:
AllElementsExceptEdgeEnrichedElements=np.setdiff1d(range(fluid_nelem1),EdgeEnrichedElements)

In [60]:
if (flag_write_gmsh_results==1) and (rank==0):
    silex_lib_gmsh.WriteResults2(results_file+'_LSenriched_elements',fluid_nodes1,fluid_elements1[LSEnrichedElements],4)
    silex_lib_gmsh.WriteResults2(results_file+'_enriched_elements',fluid_nodes1,fluid_elements1[EnrichedElements],4)
    silex_lib_gmsh.WriteResults2(results_file+'_edge_enriched_elements',fluid_nodes1,fluid_elements1[EdgeEnrichedElements],4)
    silex_lib_gmsh.WriteResults2(results_file+'_edge_enriched_elements_in_all_mesh',fluid_nodes1,fluid_elements1[EdgeEnrichedElementsInAllMesh],4)
    silex_lib_gmsh.WriteResults2(results_file+'_heaviside_enriched_elements',fluid_nodes1,fluid_elements1[HeavisideEnrichedElements],4)

if rank==0:<br>
    silex_lib_gmsh.WriteResults2(results_file+'_enriched_elements',fluid_nodes1,fluid_elements1[EnrichedElements],4)

################################################################<br>
Compute coupling STRUCTURE / AIR terms<br>
################################################################

In [61]:
tic = time.clock()

/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [62]:
IIc1,JJc1,Vc1=silex_lib_xfem_acou_tet4.computexfemcoupling1(fluid_nodes1,struc_nodes,fluid_elements1,struc_elements,EnrichedElements)
IIc2,JJc2,Vc2=silex_lib_xfem_acou_tet4.computexfemcoupling2(fluid_nodes1,struc_nodes,fluid_elements1,struc_elements,EnrichedElements,LevelSet)

In [63]:
CSA=0.5*scipy.sparse.csc_matrix( (Vc1,(IIc1,JJc1)), shape=(struc_ndof,fluid_ndof1) )+0.5*scipy.sparse.csc_matrix( (Vc2,(IIc2,JJc2)), shape=(struc_ndof,fluid_ndof1) )

In [64]:
toc = time.clock()
if rank==0:
    print ("time to compute coupling matrices:",toc-tic)

time to compute coupling matrices: 1.7374290000000006


/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


############################################################<br>
Compute Standard Fluid Matrices<br>
############################################################

In [65]:
tic = time.clock()

/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [66]:
IIf,JJf,Vffk,Vffm=silex_lib_xfem_acou_tet4.globalacousticmatrices(fluid_elements1,fluid_nodes1,celerity,rho)

In [67]:
KFF=scipy.sparse.csc_matrix( (Vffk,(IIf,JJf)), shape=(fluid_ndof1,fluid_ndof1) )
MFF=scipy.sparse.csc_matrix( (Vffm,(IIf,JJf)), shape=(fluid_ndof1,fluid_ndof1) )

In [68]:
SolvedDofF=list(range(fluid_ndof1))

############################################################<br>
Compute impedance matrices<br>
############################################################

In [69]:
print(silex_lib_porous_tet4_fortran.stiffnessimpedanceparoi.__doc__)
IIimp,JJimp,Vimp=silex_lib_porous_tet4_fortran.stiffnessimpedanceparoi(fluid_nodes1,InterfaceConnectivity)

ii,jj,vc = stiffnessimpedanceparoi(nodef,elemi,[nnodef,nelemi])

Wrapper for ``stiffnessimpedanceparoi``.

Parameters
----------
nodef : input rank-2 array('d') with bounds (nnodef,3)
elemi : input rank-2 array('i') with bounds (nelemi,3)

Other Parameters
----------------
nnodef : input int, optional
    Default: shape(nodef,0)
nelemi : input int, optional
    Default: shape(elemi,0)

Returns
-------
ii : rank-1 array('i') with bounds (9 * nelemi)
jj : rank-1 array('i') with bounds (9 * nelemi)
vc : rank-1 array('d') with bounds (9 * nelemi)



In [70]:
CII=scipy.sparse.csc_matrix( (Vimp,(IIimp,JJimp)), shape=(fluid_ndof1,fluid_ndof1) )

############################################################<br>
Compute Coupling Porous-air Matrices<br>
############################################################

rint(silex_lib_porous_tet4_fortran.computecouplingporousair.__doc__)

IIpf,JJpf,Vpf=silex_lib_porous_tet4_fortran.computecouplingporousair(fluid_nodes1,InterfaceConnectivity,po_por)<br>
CPF=scipy.sparse.csc_matrix( (Vpf,(IIpf,JJpf)), shape=(fluid_ndof2,fluid_ndof1) )

olvedDof = np.hstack([SolvedDofF,SolvedDofP+fluid_ndof1])

################################################################<br>
Compute Heaviside enrichment<br>
################################################################

In [71]:
tic = time.clock()

/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


nrichednodes = np.unique(fluid_elements1[HeavisideEnrichedElements])

In [72]:
Enrichednodes = np.unique(fluid_elements1[EnrichedElements])

In [73]:
NegativeLSelements,PositiveLSelements,NegativeLStgtElements,PositiveLStgtElements,nbNegLS,nbPosLS,nbNegLSt,nbPosLSt=silex_lib_xfem_acou_tet4.getpositivenegativeelts(fluid_elements1,LevelSet,LevelSetTangent)

In [74]:
NegativeLSelements=NegativeLSelements[list(range(nbNegLS))]
PositiveLSelements=PositiveLSelements[list(range(nbPosLS))]
NegativeLStgtElements=NegativeLStgtElements[list(range(nbNegLSt))]
PositiveLStgtElements=PositiveLStgtElements[list(range(nbPosLSt))]

Iaa,JJaa,IIaf,JJaf,Vaak,Vaam,Vafk,Vafm=silex_lib_xfem_acou_tet4.globalxfemacousticmatrices(fluid_elements1[AllElementsExceptEdge],fluid_nodes1,LevelSet,celerity,rho)

In [75]:
IIaa,JJaa,IIaf,JJaf,Vaak,Vaam,Vafk,Vafm=silex_lib_xfem_acou_tet4.globalxfemacousticmatrices(fluid_elements1[NegativeLStgtElements],fluid_nodes1,LevelSet,celerity,rho)
#IIaa,JJaa,IIaf,JJaf,Vaak,Vaam,Vafk,Vafm=silex_lib_xfem_acou_tet4.globalxfemacousticmatrices(fluid_elements1,fluid_nodes1,LevelSet,celerity,rho)

In [76]:
KAAheaviside = scipy.sparse.csc_matrix( (Vaak,(IIaa,JJaa)), shape=(fluid_ndof1,fluid_ndof1) )
MAAheaviside = scipy.sparse.csc_matrix( (Vaam,(IIaa,JJaa)), shape=(fluid_ndof1,fluid_ndof1) )
KAFheaviside = scipy.sparse.csc_matrix( (Vafk,(IIaf,JJaf)), shape=(fluid_ndof1,fluid_ndof1) )
MAFheaviside = scipy.sparse.csc_matrix( (Vafm,(IIaf,JJaf)), shape=(fluid_ndof1,fluid_ndof1) )

In [77]:
SolvedDofA=Enrichednodes-1

In [78]:
toc = time.clock()
if rank==0:
    print ("time to compute Heaviside enrichment:",toc-tic)

time to compute Heaviside enrichment: 0.14098700000000086


/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


################################################################<br>
Compute Edge enrichment<br>
################################################################

In [79]:
tic = time.clock()

/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [80]:
PartiallyPositiveLStgtElements=np.hstack([PositiveLStgtElements,EdgeEnrichedElementsInAllMesh])

I,JJ,vkaa,vmaa,vkfa,vmfa = silex_lib_xfem_acou_tet4.computeedgeenrichment(fluid_nodes1,fluid_elements1[EdgeEnrichedElements],LevelSet,LevelSetTangent,celerity,rho)

In [81]:
II,JJ,vkaa,vmaa,vkfa,vmfa = silex_lib_xfem_acou_tet4.computeedgeenrichment(fluid_nodes1,fluid_elements1[PartiallyPositiveLStgtElements],LevelSet,LevelSetTangent,celerity,rho)
#II,JJ,vkaa,vmaa,vkfa,vmfa = silex_lib_xfem_acou_tet4.computeedgeenrichment(fluid_nodes1,fluid_elements1,LevelSet,LevelSetTangent,celerity,rho)

In [82]:
KAAedge = scipy.sparse.csc_matrix( (vkaa,(II,JJ)), shape=(fluid_ndof1,fluid_ndof1) )
MAAedge = scipy.sparse.csc_matrix( (vmaa,(II,JJ)), shape=(fluid_ndof1,fluid_ndof1) )
KAFedge = scipy.sparse.csc_matrix( (vkfa,(II,JJ)), shape=(fluid_ndof1,fluid_ndof1) )
MAFedge = scipy.sparse.csc_matrix( (vmfa,(II,JJ)), shape=(fluid_ndof1,fluid_ndof1) )

In [83]:
toc = time.clock()
if rank==0:
    print ("time to compute edge enrichment:",toc-tic)

time to compute edge enrichment: 0.626315


/Users/luclaurent/Documents/1DIVERS/Tools/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [84]:
KAA=KAAheaviside+KAAedge
MAA=MAAheaviside+MAAedge
KAF=KAFheaviside+KAFedge
MAF=MAFheaviside+MAFedge

In [85]:
if (flag_write_gmsh_results==1) and (rank==0):
    silex_lib_gmsh.WriteResults2(results_file+'_NegativeLSelements',fluid_nodes1,fluid_elements1[NegativeLSelements],4)
    silex_lib_gmsh.WriteResults2(results_file+'_PositiveLSelements',fluid_nodes1,fluid_elements1[PositiveLSelements],4)
    silex_lib_gmsh.WriteResults2(results_file+'_NegativeLStgtElements',fluid_nodes1,fluid_elements1[NegativeLStgtElements],4)
    silex_lib_gmsh.WriteResults2(results_file+'_PositiveLStgtElements',fluid_nodes1,fluid_elements1[PositiveLStgtElements],4)
    silex_lib_gmsh.WriteResults2(results_file+'_PartiallyPositiveLStgtElements',fluid_nodes1,fluid_elements1[PartiallyPositiveLStgtElements],4)

################################################################<br>
Construct the whole system<br>
################################################################

To impose the load on the fluid:<br>
fluid node number 1

In [86]:
UF = np.zeros(2*fluid_ndof1+struc_ndof,dtype=float)
UF[9-1]=3.1250E-05

In [87]:
SolvedDof = np.hstack([SolvedDofF,SolvedDofA+fluid_ndof1,SolvedDofS+2*fluid_ndof1])

############################################################<br>
FRF computation<br>
############################################################

In [88]:
Flag_frf_analysis=1
frequencies=[]
frf=[]

In [89]:
if (Flag_frf_analysis==1):
    print ("Proc. ",rank," / time at the beginning of the FRF:",time.ctime())
    press_save=[]
    for i in range(nb_freq_step_per_proc):
        freq = freq_ini+i*nproc*deltafreq+rank*deltafreq
        frequencies.append(freq)
        omega=2*np.pi*freq
        print ("proc number",rank,"frequency=",freq)

        #IIp,JJp,Vppk,Vppm=silex_lib_porous_tet4_fortran.stiffnessmassmatrix(fluid_nodes2,fluid_elements2,porous_material_prop,omega)
        #KPP=scipy.sparse.csc_matrix( (Vppk,(IIp,JJp)), shape=(fluid_ndof2,fluid_ndof2) )
        #MPP=scipy.sparse.csc_matrix( (Vppm,(IIp,JJp)), shape=(fluid_ndof2,fluid_ndof2) )
        Kimp=-(omega**2/(k_imp_paroi-1j*omega*d_imp_paroi))*CII
        K=scipy.sparse.construct.bmat( [ [KFF[SolvedDofF,:][:,SolvedDofF]+Kimp[SolvedDofF,:][:,SolvedDofF],KAF[SolvedDofF,:][:,SolvedDofA],None],
                                         [KAF[SolvedDofA,:][:,SolvedDofF],KAA[SolvedDofA,:][:,SolvedDofA],None],
                                         [None,       -CSA[SolvedDofS,:][:,SolvedDofA],KSS[SolvedDofS,:][:,SolvedDofS]]
                                         ] )
        
        M=scipy.sparse.construct.bmat( [ [MFF[SolvedDofF,:][:,SolvedDofF],MAF[SolvedDofF,:][:,SolvedDofA],None],
                                         [MAF[SolvedDofA,:][:,SolvedDofF],MAA[SolvedDofA,:][:,SolvedDofA],CSA[SolvedDofS,:][:,SolvedDofA].T],
                                         [None,        None,                                  MSS[SolvedDofS,:][:,SolvedDofS]]] )
        F=np.array(omega**2*UF[SolvedDof] , dtype='c16')
        
        sol = mumps.spsolve( scipy.sparse.csc_matrix(K-(omega**2)*M,dtype='c16') , F , comm=mycomm )
        press1 = np.zeros((fluid_ndof1),dtype=complex)
        press1[SolvedDofF]=sol[list(range(len(SolvedDofF)))]
        enrichment=np.zeros((fluid_nnodes),dtype=complex)
        enrichment[SolvedDofA]=sol[list(range(len(SolvedDofF),len(SolvedDofF)+len(SolvedDofA)))]
        CorrectedPressure=press1
        CorrectedPressure[SolvedDofA]=CorrectedPressure[SolvedDofA]+enrichment[SolvedDofA]*np.sign(LevelSet[SolvedDofA])
        frf.append(silex_lib_xfem_acou_tet4.computecomplexquadratiquepressure(fluid_elements5,fluid_nodes1,CorrectedPressure))
        if (flag_write_gmsh_results==1) and (rank==0):
            press_save.append(CorrectedPressure.real)
            
    frfsave=[frequencies,frf]
    if rank!=0:
        comm.send(frfsave, dest=0, tag=11)
    if (flag_write_gmsh_results==1) and (rank==0):
        silex_lib_gmsh.WriteResults2(results_file+str(rank)+'_results_fluid_frf',fluid_nodes1,fluid_elements1,4,[[press_save,'nodal',1,'pressure']])
    print ("Proc. ",rank," / time at the end of the FRF:",time.ctime())

    # Save the FRF problem
    Allfrequencies=np.zeros(nb_freq_step)
    Allfrf=np.zeros(nb_freq_step)
    k=0
    if rank==0:
        for i in range(nproc):
            if i==0:
                data = frfsave
            else:
                data = comm.recv(source=i, tag=11)
            for j in range(len(data[0])):
                Allfrequencies[k]=data[0][j]
                Allfrf[k]=data[1][j]
                k=k+1
        Allfrequencies, Allfrf = zip(*sorted(zip(Allfrequencies, Allfrf)))
        Allfrfsave=[np.array(list(Allfrequencies)),np.array(list(Allfrf))]
        f=open(results_file+'_results.frf','wb')
        pickle.dump(Allfrfsave, f)
        f.close()
        print('nb of total dofs: ',len(SolvedDofF)+len(SolvedDofA)+len(SolvedDofS),)

Proc.  0  / time at the beginning of the FRF: Wed Apr 13 10:20:18 2022
proc number 0 frequency= 10.0
proc number 0 frequency= 10.237797246558198
proc number 0 frequency= 10.475594493116395
proc number 0 frequency= 10.713391739674593
proc number 0 frequency= 10.951188986232792
proc number 0 frequency= 11.188986232790988
proc number 0 frequency= 11.426783479349186
proc number 0 frequency= 11.664580725907385
proc number 0 frequency= 11.902377972465581
proc number 0 frequency= 12.14017521902378
proc number 0 frequency= 12.377972465581978
proc number 0 frequency= 12.615769712140175
proc number 0 frequency= 12.853566958698373
proc number 0 frequency= 13.091364205256571
proc number 0 frequency= 13.32916145181477
proc number 0 frequency= 13.566958698372966
proc number 0 frequency= 13.804755944931165
proc number 0 frequency= 14.042553191489361
proc number 0 frequency= 14.28035043804756
proc number 0 frequency= 14.518147684605758
proc number 0 frequency= 14.755944931163956
proc number 0 frequenc